In [1]:
################################################################################
# Author 1:      Bernhard Lugger
# MatNr 1:       01114792
# Author 2:      Paula Nauta
# MatNr 2:       11938311
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [16]:
# imports
import matplotlib.pyplot as plt # or from matplotlib import pyplot as plt
import matplotlib.dates as mdates # we use this to space our date-xticks
import numpy as np
import pandas as pd

import plotly.express as px
# Plotly express is especcialy useful if you want to plot dataframes (e.g. Pandas Dataframe).
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [59]:
# -------- functions ------------

def load_oecd_data_to_df(file_path):
    """loads one csv from the given path and returns the 3 relevant columns to a dataframe."""
    oecd_data = pd.read_csv(file_path, header=0, usecols=['LOCATION', 'TIME', 'Value'], delimiter=",")
    return oecd_data


def choosen_countries(raw_df, *countries):   # z.b 'AUS', 'BEL'
    """ausgewählte länder aus dem dataframe in ein neues dataframe"""
    country_names = []
    for country in countries:
        country_names += country

    #TODO opitionl, nicht notwenig:
    # - zeilen: where choosen countries, zB.
    return country_names


def plot_lineplot(df, *countries):
    """plots a plot..."""
    country_names = []
    for country in countries:
        country_names += country
    #print(country_names)

    data_test = df.loc[df.LOCATION == country_names[1]]
    fig = px.line(data_test, x="TIME",y="Value", title="bla")
    fig.show()

    # entweder raw_df einlesen und nur einzelne Länder ausschneiden/verwenden und ploten
    # ODER
    # vorbereitetes cropped_df ploten <-- wirkt sinnvoller für diese funktion


def plot_barplot(df, country1, country2, country3, country4, country5):
    return 0


def plot_scatterplot(df, country1, country2):
    return 0

In [60]:
# ------------------ Test -------------------------
path_to_file = 'datasets/original/DP_LIVE_15122021160749354.csv'
df1 = load_oecd_data_to_df(path_to_file)
countries = ("AUS", "BEL")

plot_lineplot(df1, countries)

['AUS', 'BEL']


In [ ]:
# ------------ final program (later) ----------
"""

plot_lineplot(land_a, land_b)
plot_lineplot(land_a, land_b)
plot_lineplot(land_a, land_b)

bar_lineplot(country1, country2, country3, country4, country5)
bar_lineplot(country1, country2, country3)
bar_lineplot(country1, country2, country3)

scatter
scatter
scatter

"""
